In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import os.path
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [25]:
year1 = 18
year2 = 19

pd.set_option('display.max_columns', 500)

base_url = ('https://stats.nba.com/teams/boxscores/?Season=20{}-{}&SeasonType=Regular%20Season')

driver = webdriver.Chrome('/Users/jordanlevy/Downloads/chromedriver')

In [26]:
local_filename = 'NBA_Team_Data_{}-{}'

In [28]:
# driver.get('http://www.google.com')

In [29]:
### Download File
driver.get(base_url.format(year1,year2))
print('Got url')


Got url


In [17]:
import time

In [30]:
# Finds page selector, and clicks on 'All' (this can be done manually if needed)
page_selector = driver.find_elements_by_tag_name('select')[2]
page_selector.click()
page_selector.find_elements_by_tag_name("option")[0].click()

In [31]:
%time soup = BeautifulSoup(driver.page_source, 'html5lib')

CPU times: user 25.3 s, sys: 397 ms, total: 25.7 s
Wall time: 27.3 s


In [32]:
nba_table = soup.find('div', {'class':'nba-stat-table'})

In [33]:
type(nba_table)

bs4.element.Tag

In [34]:
team_columns = []
for row in nba_table.find('thead').find_all('th'):
    #print(row.text)
    team_columns.append(row.text)
team_columns

['Team',
 'Match\xa0Up',
 '\n            Game\xa0Date\n          ',
 '\n            Season\n          ',
 'W/L',
 '\n            MIN\n          ',
 '\n            PTS\n          ',
 '\n            FGM\n          ',
 '\n            FGA\n          ',
 '\n            FG%\n          ',
 '\n            3PM\n          ',
 '\n            3PA\n          ',
 '\n            3P%\n          ',
 '\n            FTM\n          ',
 '\n            FTA\n          ',
 '\n            FT%\n          ',
 '\n            OREB\n          ',
 '\n            DREB\n          ',
 '\n            REB\n          ',
 '\n            AST\n          ',
 '\n            STL\n          ',
 '\n            BLK\n          ',
 '\n            TOV\n          ',
 '\n            PF\n          ',
 '\n            +/-\n          ']

In [35]:
for row in range(len(team_columns)):
    team_columns[row] = team_columns[row].strip(' ').strip('\n').strip(' ').replace('\xa0', ' ')
team_columns

['Team',
 'Match Up',
 'Game Date',
 'Season',
 'W/L',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG%',
 '3PM',
 '3PA',
 '3P%',
 'FTM',
 'FTA',
 'FT%',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 '+/-']

In [36]:
%%time 
all_rows = nba_table.find('tbody').find_all('tr')
team_data = []
for row in all_rows:
    row_values = []
    for val in row.find_all('td'):
        row_values.append(val.text.strip(' ').strip('\n').strip(' '))
    team_data.append(row_values)
team_data

CPU times: user 442 ms, sys: 4.87 ms, total: 446 ms
Wall time: 447 ms


[['MEM',
  'MEM vs. GSW',
  '04/10/2019',
  '',
  'W',
  '240',
  '132',
  '48',
  '98',
  '49.0',
  '21',
  '46',
  '45.7',
  '15',
  '18',
  '83.3',
  '15',
  '34',
  '49',
  '25',
  '9',
  '0',
  '11',
  '18',
  '15'],
 ['GSW',
  'GSW @ MEM',
  '04/10/2019',
  '',
  'L',
  '240',
  '117',
  '46',
  '92',
  '50.0',
  '13',
  '30',
  '43.3',
  '12',
  '14',
  '85.7',
  '9',
  '30',
  '39',
  '32',
  '3',
  '5',
  '14',
  '17',
  '-15'],
 ['ORL',
  'ORL @ CHA',
  '04/10/2019',
  '',
  'W',
  '240',
  '122',
  '48',
  '88',
  '54.5',
  '11',
  '30',
  '36.7',
  '15',
  '20',
  '75.0',
  '9',
  '30',
  '39',
  '24',
  '5',
  '2',
  '6',
  '19',
  '8'],
 ['CHA',
  'CHA vs. ORL',
  '04/10/2019',
  '',
  'L',
  '240',
  '114',
  '41',
  '78',
  '52.6',
  '8',
  '29',
  '27.6',
  '24',
  '27',
  '88.9',
  '4',
  '28',
  '32',
  '21',
  '3',
  '4',
  '8',
  '14',
  '-8'],
 ['MIN',
  'MIN @ DEN',
  '04/10/2019',
  '',
  'L',
  '240',
  '95',
  '39',
  '91',
  '42.9',
  '13',
  '32',
  '40.6',


In [37]:
team_data_df = pd.DataFrame(data=team_data, columns=team_columns)
team_data_df.head()

,Team,Match Up,Game Date,Season,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,MEM,MEM vs. GSW,04/10/2019,,W,240,132,48,98,49.0,21,46,45.7,15,18,83.3,15,34,49,25,9,0,11,18,15
1,GSW,GSW @ MEM,04/10/2019,,L,240,117,46,92,50.0,13,30,43.3,12,14,85.7,9,30,39,32,3,5,14,17,-15
2,ORL,ORL @ CHA,04/10/2019,,W,240,122,48,88,54.5,11,30,36.7,15,20,75.0,9,30,39,24,5,2,6,19,8
3,CHA,CHA vs. ORL,04/10/2019,,L,240,114,41,78,52.6,8,29,27.6,24,27,88.9,4,28,32,21,3,4,8,14,-8
4,MIN,MIN @ DEN,04/10/2019,,L,240,95,39,91,42.9,13,32,40.6,4,6,66.7,7,34,41,24,6,0,10,22,-4


We drop the +/- column to not give away who wins.

In [38]:
team_data_df = team_data_df.drop('+/-', axis=1)

In [39]:
team_data_df.to_csv('NBA_Team_Data_{}-{}.csv'.format(year1, year2), index=False)